In [1]:
from networkx import DiGraph, set_node_attributes
from vrpy import VehicleRoutingProblem
from itertools import chain, combinations

In [14]:
G = DiGraph()
for v in [1, 2, 3, 4, 5]:
    G.add_edge("Source", v, cost=10)
    G.add_edge(v, "Sink", cost=10)
G.add_edge(1, 2, cost=10)
G.add_edge(2, 3, cost=10)
G.add_edge(3, 4, cost=15)
G.add_edge(4, 5, cost=10)

In [15]:
prob = VehicleRoutingProblem(G)

1. Capacidade dos Veículos: Cada veículo tem uma capacidade máxima de carga que não pode ser excedida.
2. Demanda dos Clientes: Cada cliente tem uma demanda específica que deve ser completamente atendida por um único veículo.
3. Custo da Rota: Cada rota possui um custo, que simboliza o cômputo agregado da distância percorrida, do tempo de entrega, e do consumo de combustível. O objetivo é minimizar o custo total.
4. Número máximo de visitas por rota: Cada rota pode visitar um número máximo de cidades.
5. Ponto de Partida e Chegada: Todos os veículos começam e terminam suas rotas no depósito da empresa.

![img](imgs/nodes_ex1.png)

In [16]:
prob.num_stops = 2
prob.num_vehicles = 3
prob.solve()

INFO:vrpy.vrp:Clarke & Wright solution found with value 80 and 3 vehicles
INFO:vrpy.vrp:Greedy solution found with value 85 and 3 vehicles
INFO:vrpy.vrp:iteration 0, 80.0
INFO:vrpy.vrp:iteration 1, 80.0
INFO:vrpy.vrp:iteration 2, 80.0
INFO:vrpy.master_solve_pulp:total cost = 80.0


In [17]:
prob.best_routes

{1: ['Source', 1, 2, 'Sink'],
 2: ['Source', 3, 'Sink'],
 3: ['Source', 4, 5, 'Sink']}

### Add Capacity Constraints 

In [18]:
G = DiGraph()
for v in [1, 2, 3, 4, 5]:
    G.add_edge("Source", v, cost=10) # add constrain 3
    G.add_edge(v, "Sink", cost=10)
G.add_edge(1, 2, cost=10)
G.add_edge(2, 3, cost=10)
G.add_edge(3, 4, cost=15)
G.add_edge(4, 5, cost=10)

prob = VehicleRoutingProblem(G)
# prob.num_stops = 3 # Add constrain 4
for v in G.nodes():
    if v not in ["Source", "Sink"]:
        G.nodes[v]["demand"] = 5 # Add constrain 2
prob.load_capacity = 10 # Add constrain 1


In [19]:
prob.solve()
prob.best_value

INFO:vrpy.vrp:new upper bound : max num stops = 4
INFO:vrpy.vrp:Clarke & Wright solution found with value 80 and 3 vehicles
INFO:vrpy.vrp:Greedy solution found with value 85 and 3 vehicles
INFO:vrpy.vrp:iteration 0, 80.0
INFO:vrpy.vrp:iteration 1, 80.0
INFO:vrpy.vrp:iteration 2, 80.0
INFO:vrpy.vrp:iteration 3, 80.0
INFO:vrpy.master_solve_pulp:total cost = 80.0


80

In [20]:
prob.best_routes

{1: ['Source', 1, 2, 'Sink'],
 2: ['Source', 4, 5, 'Sink'],
 3: ['Source', 3, 'Sink']}

## Padrão da disciplina

In [21]:
import random

def gerar_dicionario_demandas(N):
    """
    Gera um dicionário onde a chave é um int de 1 até N e o valor é um inteiro aleatório de 1 até 10.

    :param N: Número máximo para as chaves do dicionário.
    :return: Dicionário com chaves de 1 até N e valores inteiros aleatórios de 1 até 10.
    """
    return {i: random.randint(1, 10) for i in range(1, N)}


def gerar_entradas_grafo(num_nos, max_peso=100, probabilidade=0.25):
    """
    Gera um grafo para o problema de otimização de rotas de veículos.

    :param num_nos: Número de nós no grafo, incluindo o depósito.
    :param max_peso: Peso máximo para as arestas do grafo.
    :param probabilidade: Probabilidade de criar uma rota entre duas cidades.
    :return: Um dicionário representando o grafo onde as chaves são tuplas representando as arestas (nó1, nó2)
             e os valores são os pesos dessas arestas.
    """
    grafo = {}
    # Gerar pesos para arestas entre o depósito e outros nós
    for i in range(1, num_nos):
        grafo[(0, i)] = random.randint(1, max_peso)
        grafo[(i, 0)] = grafo[(0, i)]  # Assume que a distância de volta ao depósito é a mesma

    # Gerar pesos para arestas entre todos os outros pares de nós
    for i in range(1, num_nos+1):
        for j in range(i+1, num_nos):
            if random.random() > (1 - probabilidade):  # Verifica a probabilidade
                peso = random.randint(1, max_peso)
                grafo[(i, j)] = peso

    return grafo

In [23]:
random.seed(42)
num_nos = 11                                   # Número total de nós incluindo o depósito
demandas = gerar_dicionario_demandas(num_nos)  # Gera as demandas para cada nó
grafo = gerar_entradas_grafo(num_nos)          # Gera o grafo que representa os locais e custos entre eles


In [24]:
demandas, grafo

({1: 2, 2: 1, 3: 5, 4: 4, 5: 4, 6: 3, 7: 2, 8: 9, 9: 2, 10: 10},
 {(0, 1): 55,
  (1, 0): 55,
  (0, 2): 5,
  (2, 0): 5,
  (0, 3): 4,
  (3, 0): 4,
  (0, 4): 12,
  (4, 0): 12,
  (0, 5): 28,
  (5, 0): 28,
  (0, 6): 30,
  (6, 0): 30,
  (0, 7): 65,
  (7, 0): 65,
  (0, 8): 78,
  (8, 0): 78,
  (0, 9): 4,
  (9, 0): 4,
  (0, 10): 72,
  (10, 0): 72,
  (1, 7): 1,
  (1, 8): 21,
  (2, 4): 44,
  (3, 6): 11,
  (3, 8): 80,
  (3, 9): 47,
  (6, 9): 83,
  (9, 10): 89})

In [25]:

# Salva o grafo em um arquivo TXT
with open('grafo.txt', 'w') as arquivo:
  arquivo.write(str(num_nos) + "\n")    # Número de nós, incluindo depósito
  for local, demanda in demandas.items():
    linha = f"{local} {demanda}\n"      # Par LOCAL DEMANDA
    arquivo.write(linha)

  arquivo.write(str(len(grafo)) + "\n") # Número de arestas
  for aresta, peso in grafo.items():
    linha = f"{aresta[0]} {aresta[1]} {peso}\n" # Trio: ORIGEM DESTINO CUSTO
    arquivo.write(linha)

In [26]:
def ler_arquivo_grafo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        # Lê o número de nós
        N = int(arquivo.readline().strip())-1

        # Lê as demandas dos nós
        demandas = {}
        for _ in range(N):
            linha = arquivo.readline().strip().split()
            id_no, demanda = int(linha[0]), int(linha[1])
            demandas[id_no] = demanda

        # Lê o número de arestas
        K = int(arquivo.readline().strip())

        # Lê as arestas
        arestas = []
        for _ in range(K):
            linha = arquivo.readline().strip().split()
            origem, destino, peso = int(linha[0]), int(linha[1]), int(linha[2])
            arestas.append((origem, destino, peso))

    return demandas, arestas

In [35]:
caminho_arquivo = './data/grafo_11.txt'
demandas, arestas = ler_arquivo_grafo(caminho_arquivo)

In [45]:
demandas, arestas

({1: 2, 2: 1, 3: 5, 4: 4, 5: 4, 6: 3, 7: 2, 8: 9, 9: 2, 10: 10},
 [(0, 1, 55),
  (1, 0, 55),
  (0, 2, 5),
  (2, 0, 5),
  (0, 3, 4),
  (3, 0, 4),
  (0, 4, 12),
  (4, 0, 12),
  (0, 5, 28),
  (5, 0, 28),
  (0, 6, 30),
  (6, 0, 30),
  (0, 7, 65),
  (7, 0, 65),
  (0, 8, 78),
  (8, 0, 78),
  (0, 9, 4),
  (9, 0, 4),
  (0, 10, 72),
  (10, 0, 72),
  (1, 7, 1),
  (1, 8, 21),
  (2, 4, 44),
  (3, 6, 11),
  (3, 8, 80),
  (3, 9, 47),
  (6, 9, 83),
  (9, 10, 89)])

In [60]:
G = DiGraph()
for inicio, fim, custo in arestas:
    if inicio==0: inicio="Source"
    if fim==0: fim="Sink"
    G.add_edge(inicio, fim, cost=custo)

In [61]:
set_node_attributes(G, values=demandas, name="demand")

In [63]:
prob = VehicleRoutingProblem(G)  # Pode alterar a capacidade
prob.load_capacity = 15
prob.num_stops = 5                                 # Pode alterar o número máximo de paradas
prob.solve()

INFO:vrpy.vrp:new upper bound : max num stops = 5
INFO:vrpy.vrp:Clarke & Wright solution found with value 604 and 6 vehicles
INFO:vrpy.vrp:Greedy solution found with value 640 and 6 vehicles
INFO:vrpy.vrp:iteration 0, 602.0
INFO:vrpy.vrp:iteration 1, 589.0
INFO:vrpy.vrp:iteration 2, 589.0
INFO:vrpy.vrp:iteration 3, 562.0
INFO:vrpy.master_solve_pulp:total cost = 564.0


In [64]:
prob.best_routes

{1: ['Source', 8, 'Sink'],
 2: ['Source', 1, 7, 'Sink'],
 3: ['Source', 5, 'Sink'],
 4: ['Source', 3, 6, 'Sink'],
 5: ['Source', 10, 'Sink'],
 6: ['Source', 9, 'Sink'],
 7: ['Source', 4, 'Sink'],
 8: ['Source', 2, 'Sink']}

In [50]:
prob.best_value

564

In [51]:
prob.best_routes_cost

{1: 156, 2: 56, 3: 121, 4: 45, 5: 144, 6: 8, 7: 24, 8: 10}

In [52]:
prob.best_routes_load

{1: 9, 2: 4, 3: 4, 4: 8, 5: 10, 6: 2, 7: 4, 8: 1}

### Python version

In [272]:
capacidade = 15

In [273]:
def to_matrix_adj(arestas):
    matrix = {'sink': set(), 'source': set()}
    for a in arestas:
        origem, destino = a[0], a[1]
        if origem == 0:
            origem = 'source'
        if destino == 0:
            destino = 'sink'
        if origem not in matrix:
            matrix[origem] = set()
        matrix[origem].add(destino)
    return matrix

def find_all_paths(graph):
    if graph == [] or graph == {}: return []
    source, destine = 'source', 'sink'
    queue, paths = [[source]], []
    count = 0
    while queue:
        count += 1
        path = queue.pop(0)
        node = path[-1]
        if node == destine:
            paths.append(path)
        for neighbor in graph[node]:
            queue.append(path.copy() + [neighbor])
    return paths

def gerar_todas_combinacoes(arestas):
    matrix_adj = to_matrix_adj(arestas)
    all_paths = find_all_paths(matrix_adj)
    return all_paths

def checa_capacidade(combinacao, demanda, capacidade):
    carga_total = 0
    for local in combinacao:
        if local != 'source' and local != 'sink':
            carga_total += demanda[local]
    if carga_total <= capacidade:
        return True
    return False

def calcula_custo(combinacao, arestas):
    grafo = {(x[0], x[1]): x[2] for x in arestas}
    custo_total = 0
    for i in range(len(combinacao)-1):
        origem = combinacao[i] if combinacao[i] != 'source' and combinacao[i] != 'sink' else 0
        destino = combinacao[i+1] if combinacao[i+1] != 'source' and combinacao[i+1] != 'sink' else 0
        custo_total += grafo[(origem, destino)]
    return custo_total

graph = to_matrix_adj(arestas)
all_paths = find_all_paths(graph)
all_paths, graph

([['source', 1, 'sink'],
  ['source', 2, 'sink'],
  ['source', 3, 'sink'],
  ['source', 4, 'sink'],
  ['source', 5, 'sink'],
  ['source', 6, 'sink'],
  ['source', 7, 'sink'],
  ['source', 8, 'sink'],
  ['source', 9, 'sink'],
  ['source', 1, 8, 'sink'],
  ['source', 1, 9, 'sink'],
  ['source', 2, 6, 'sink'],
  ['source', 3, 9, 'sink'],
  ['source', 4, 6, 'sink'],
  ['source', 4, 7, 'sink']],
 {'sink': set(),
  'source': {1, 2, 3, 4, 5, 6, 7, 8, 9},
  1: {8, 9, 'sink'},
  2: {6, 'sink'},
  3: {9, 'sink'},
  4: {6, 7, 'sink'},
  5: {'sink'},
  6: {'sink'},
  7: {'sink'},
  8: {'sink'},
  9: {'sink'}})

In [274]:
# Solver global search VRP from scratch
def solver_vrp(arestas, demanda, capacidade):
    melhor_rota = None
    melhor_custo = float('inf')
    rotas = []
    combinacoes = gerar_todas_combinacoes(arestas)
    
    for comb in combinacoes:
        if checa_capacidade(comb, demanda, capacidade):
            custo = calcula_custo(comb, arestas)
            rotas.append({'path': comb, 'cost': custo})
            if custo < melhor_custo:
                melhor_custo = custo
                melhor_rota = comb
                
    
    
    return melhor_rota, melhor_custo, rotas
    

In [275]:
def power_set(iterable):
    "power_set([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

list(power_set([1, 2, 3]))

[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

## Checking

In [276]:
[checa_capacidade(x, demandas, capacidade=5) for x in all_paths]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False]

In [277]:
[calcula_custo(x, arestas) for x in all_paths]

[152, 110, 10, 8, 24, 56, 60, 130, 156, 142, 175, 127, 94, 112, 81]

In [278]:
gerar_todas_combinacoes(arestas)

[['source', 1, 'sink'],
 ['source', 2, 'sink'],
 ['source', 3, 'sink'],
 ['source', 4, 'sink'],
 ['source', 5, 'sink'],
 ['source', 6, 'sink'],
 ['source', 7, 'sink'],
 ['source', 8, 'sink'],
 ['source', 9, 'sink'],
 ['source', 1, 8, 'sink'],
 ['source', 1, 9, 'sink'],
 ['source', 2, 6, 'sink'],
 ['source', 3, 9, 'sink'],
 ['source', 4, 6, 'sink'],
 ['source', 4, 7, 'sink']]

In [279]:
melhor_rota, melhor_custo, all_paths = solver_vrp(arestas, demandas, capacidade=15)

In [280]:
def total_cost(comb):
    return sum([x['cost'] for x in comb])

def get_best_route(all_paths):
    power_set_paths = list(power_set(all_paths))
    valid_combinations = [x for x in power_set_paths if checa_comb_com_demanda(x, demandas)]
    return sorted(valid_combinations, key=total_cost)[0]

def checa_comb_com_demanda(comb, demanda):
    if len(comb) < 1: return False
    demanda_keys = list(demanda.keys())
    buffer_demanda = demanda_keys.copy()
    for x in comb:
        path = x['path']
        for d in buffer_demanda:
            if d in path and d in demanda_keys:
                demanda_keys.remove(d)
        if len(demanda_keys) == 0:
            return True
    return False

In [281]:
best_route = get_best_route(all_paths)
best_route

({'path': ['source', 4, 'sink'], 'cost': 8},
 {'path': ['source', 5, 'sink'], 'cost': 24},
 {'path': ['source', 7, 'sink'], 'cost': 60},
 {'path': ['source', 1, 8, 'sink'], 'cost': 142},
 {'path': ['source', 2, 6, 'sink'], 'cost': 127},
 {'path': ['source', 3, 9, 'sink'], 'cost': 94})

### Final Version:

In [2]:
import random

def gerar_dicionario_demandas(N):
    """
    Gera um dicionário onde a chave é um int de 1 até N e o valor é um inteiro aleatório de 1 até 10.

    :param N: Número máximo para as chaves do dicionário.
    :return: Dicionário com chaves de 1 até N e valores inteiros aleatórios de 1 até 10.
    """
    return {i: random.randint(1, 10) for i in range(1, N)}


def gerar_entradas_grafo(num_nos, max_peso=100, probabilidade=0.25):
    """
    Gera um grafo para o problema de otimização de rotas de veículos.

    :param num_nos: Número de nós no grafo, incluindo o depósito.
    :param max_peso: Peso máximo para as arestas do grafo.
    :param probabilidade: Probabilidade de criar uma rota entre duas cidades.
    :return: Um dicionário representando o grafo onde as chaves são tuplas representando as arestas (nó1, nó2)
             e os valores são os pesos dessas arestas.
    """
    grafo = {}
    # Gerar pesos para arestas entre o depósito e outros nós
    for i in range(1, num_nos):
        grafo[(0, i)] = random.randint(1, max_peso)
        grafo[(i, 0)] = grafo[(0, i)]  # Assume que a distância de volta ao depósito é a mesma

    # Gerar pesos para arestas entre todos os outros pares de nós
    for i in range(1, num_nos+1):
        for j in range(i+1, num_nos):
            if random.random() > (1 - probabilidade):  # Verifica a probabilidade
                peso = random.randint(1, max_peso)
                grafo[(i, j)] = peso

    return grafo

def ler_arquivo_grafo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        # Lê o número de nós
        N = int(arquivo.readline().strip())-1

        # Lê as demandas dos nós
        demandas = {}
        for _ in range(N):
            linha = arquivo.readline().strip().split()
            id_no, demanda = int(linha[0]), int(linha[1])
            demandas[id_no] = demanda

        # Lê o número de arestas
        K = int(arquivo.readline().strip())

        # Lê as arestas
        arestas = []
        for _ in range(K):
            linha = arquivo.readline().strip().split()
            origem, destino, peso = int(linha[0]), int(linha[1]), int(linha[2])
            arestas.append((origem, destino, peso))

    return demandas, arestas


def generate_batch_samples(n):
    for i in range(4, n + 4, 1):
        random.seed(42)
        
        num_nos = i                                   # Número total de nós incluindo o depósito
        demandas = gerar_dicionario_demandas(num_nos)  # Gera as demandas para cada nó
        grafo = gerar_entradas_grafo(num_nos)          # Gera o grafo que representa os locais e custos entre eles
        filename = f'./data/grafo_{i}.txt'
        # Salva o grafo em um arquivo TXT
        with open(filename, 'w') as arquivo:
            arquivo.write(str(num_nos) + "\n")    # Número de nós, incluindo depósito
            for local, demanda in demandas.items():
                linha = f"{local} {demanda}\n"      # Par LOCAL DEMANDA
                arquivo.write(linha)

            arquivo.write(str(len(grafo)) + "\n") # Número de arestas
            for aresta, peso in grafo.items():
                linha = f"{aresta[0]} {aresta[1]} {peso}\n" # Trio: ORIGEM DESTINO CUSTO
                arquivo.write(linha)

# generate_batch_samples(10)


In [93]:
a= [5, 4, 2 ]
b = sorted(a)
b

[2, 4, 5]

In [ ]:
def to_matrix_adj(arestas):
    matrix = {'sink': set(), 'source': set()}
    for a in arestas:
        origem, destino = a[0], a[1]
        if origem == 0:
            origem = 'source'
        if destino == 0:
            destino = 'sink'
        if origem not in matrix:
            matrix[origem] = set()
        matrix[origem].add(destino)
    return matrix

# def find_all_paths(graph, demanda=None, arestas=None, bfs=False):
#     if bfs:
#         if graph == [] or graph == {}: return []
#         source, destine = 'source', 'sink'
#         queue, paths = [[source]], []
#         while queue:
#             path = queue.pop(0)
#             node = path[-1]
#             if node == destine:
#                 paths.append(path)
#             for neighbor in graph[node]:
#                 queue.append(path.copy() + [neighbor])
#     else:
#         if graph == [] or graph == {}: return []
#         source, destine = 'source', 'sink'
#         queue, paths = [[source, 0]], []
#         while queue:
#             path = queue.pop(0)
#             node = path[-1]
#             if node == destine:
#                 paths.append(path)
#             sorted_neighbors = sorted(graph[node], key=lambda x: arestas.get((node, x), float('inf')))
#             for neighbor in sorted_neighbors:
#                 queue.append(path.copy() + [neighbor])
#     return paths

def find_all_paths(graph):
    if graph == [] or graph == {}: return []
    source, destine = 'source', 'sink'
    queue, paths = [[source]], []
    while queue:
        path = queue.pop(0)
        node = path[-1]
        if node == destine:
            paths.append(path)
        for neighbor in graph[node]:
            queue.append(path.copy() + [neighbor])
    print(paths)
    return paths

def gerar_todos_os_paths(arestas):
    matrix_adj = to_matrix_adj(arestas)
    all_paths = find_all_paths(matrix_adj)
    return all_paths

def checa_capacidade(path, demanda, capacidade):
    carga_total = 0
    for node in path:
        if node != 'source' and node != 'sink':
            carga_total += demanda[node]
    if carga_total <= capacidade:
        return True
    return False

def calcula_custo(path, arestas):
    grafo = {(x[0], x[1]): x[2] for x in arestas}
    custo_total = 0
    for i in range(len(path)-1):
        origem = path[i] if path[i] != 'source' and path[i] != 'sink' else 0
        destino = path[i+1] if path[i+1] != 'source' and path[i+1] != 'sink' else 0
        custo_total += grafo[(origem, destino)]
    return custo_total

def power_set(iterable):
    "power_set([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def total_cost(comb):
    return sum([x['cost'] for x in comb])

def get_best_route(all_paths):
    power_set_paths = list(power_set(all_paths))
    print(power_set_paths)
    valid_combinations = [x for x in power_set_paths if checa_comb_com_demanda(x, demandas)]
    return sorted(valid_combinations, key=total_cost)[0]

def checa_comb_com_demanda(comb, demanda):
    if len(comb) < 1: return False
    demanda_keys = list(demanda.keys())
    buffer_demanda = demanda_keys.copy()
    for x in comb:
        path = x['path']
        for d in buffer_demanda:
            if d in path and d in demanda_keys:
                demanda_keys.remove(d)
        if len(demanda_keys) == 0:
            return True
    return False

def get_all_paths(arestas, demanda, capacidade):
    all_valid_paths = []
    all_paths = gerar_todos_os_paths(arestas)
    for path in all_paths:
        if checa_capacidade(path, demanda, capacidade):
            custo = calcula_custo(path, arestas)
            all_valid_paths.append({'path': path, 'cost': custo})
    return all_valid_paths

def solver_vrp(arestas, demanda, capacidade):
    all_paths = get_all_paths(arestas, demanda, capacidade)
    best_route = get_best_route(all_paths)
    return best_route

In [4]:
capacidade = 15

In [33]:
# all_solutions = []
# for i in range(4, 12, 1):
#     caminho_arquivo = f'./data/grafo_{i}.txt'
#     demandas, arestas = ler_arquivo_grafo(caminho_arquivo)
#     best_route = solver_vrp(arestas, demandas, capacidade)
#     best_route, total_cost(best_route)
#     print(caminho_arquivo, best_route)
#     all_solutions.append(best_route)

./data/grafo_4.txt ({'path': ['source', 1, 'sink'], 'cost': 64}, {'path': ['source', 2, 3, 'sink'], 'cost': 59})
./data/grafo_5.txt ({'path': ['source', 2, 'sink'], 'cost': 36}, {'path': ['source', 4, 'sink'], 'cost': 28}, {'path': ['source', 1, 3, 'sink'], 'cost': 136})
./data/grafo_6.txt ({'path': ['source', 3, 'sink'], 'cost': 28}, {'path': ['source', 4, 'sink'], 'cost': 174}, {'path': ['source', 5, 'sink'], 'cost': 190}, {'path': ['source', 1, 2, 'sink'], 'cost': 125})
./data/grafo_7.txt ({'path': ['source', 1, 'sink'], 'cost': 190}, {'path': ['source', 2, 'sink'], 'cost': 28}, {'path': ['source', 3, 6, 'sink'], 'cost': 197}, {'path': ['source', 4, 5, 'sink'], 'cost': 255})
./data/grafo_8.txt ({'path': ['source', 1, 'sink'], 'cost': 174}, {'path': ['source', 4, 'sink'], 'cost': 24}, {'path': ['source', 5, 'sink'], 'cost': 152}, {'path': ['source', 2, 6, 'sink'], 'cost': 151}, {'path': ['source', 3, 7, 'sink'], 'cost': 119})
./data/grafo_9.txt ({'path': ['source', 1, 'sink'], 'cost'

In [24]:
caminho_arquivo = f'./data/grafo_4.txt'
demandas, arestas = ler_arquivo_grafo(caminho_arquivo)
best_route = solver_vrp(arestas, demandas, capacidade=15)
best_route, total_cost(best_route)


[['source', 1, 'sink'], ['source', 2, 'sink'], ['source', 3, 'sink'], ['source', 2, 3, 'sink']]


(({'path': ['source', 1, 'sink'], 'cost': 64},
  {'path': ['source', 2, 3, 'sink'], 'cost': 59}),
 123)

In [ ]:
caminho_arquivo = f'./data/grafo_11.txt'
demandas, arestas = ler_arquivo_grafo(caminho_arquivo)
best_route = solver_vrp(arestas, demandas, capacidade)
best_route, total_cost(best_route)


(({'path': ['source', 2, 'sink'], 'cost': 10},
  {'path': ['source', 4, 'sink'], 'cost': 24},
  {'path': ['source', 5, 'sink'], 'cost': 56},
  {'path': ['source', 9, 'sink'], 'cost': 8},
  {'path': ['source', 10, 'sink'], 'cost': 144},
  {'path': ['source', 1, 8, 'sink'], 'cost': 154},
  {'path': ['source', 1, 7, 'sink'], 'cost': 121},
  {'path': ['source', 3, 6, 'sink'], 'cost': 45}),
 562)

In [59]:
prob.best_routes, prob.best_routes_cost, prob.best_value

({1: ['Source', 8, 'Sink'],
  2: ['Source', 5, 'Sink'],
  3: ['Source', 1, 7, 'Sink'],
  4: ['Source', 3, 6, 'Sink'],
  5: ['Source', 10, 'Sink'],
  6: ['Source', 9, 'Sink'],
  7: ['Source', 4, 'Sink'],
  8: ['Source', 2, 'Sink']},
 {1: 156, 2: 56, 3: 121, 4: 45, 5: 144, 6: 8, 7: 24, 8: 10},
 564)

## C++ Version

In [ ]:
%%writefile brute_force.cpp
#include <iostream>
#include <vector>
#include <set>
#include <map>
#include <queue>
#include <utility> // Para std::pair

// Define o tipo Edge como uma tupla de três inteiros (origem, destino, custo)
using Edge = std::tuple<int, int, int>;

// Define um grafo como um mapa de inteiros para conjuntos de inteiros
using Graph = std::map<int, std::set<int>>;

// Define um caminho como um vetor de inteiros
using Path = std::vector<int>;

// Define uma demanda como um mapa de inteiros para inteiros
using Demand = std::map<int, int>;

// Define arestas como um mapa de pares de inteiros para inteiros
using Edges = std::map<std::pair<int, int>, int>;